In [ ]:
import tensorflow as tf


tf.Tensor()


beta1 = tf.Variable(1. , tf.dtypes.float16 ,shape=tf.TensorShape(None))
beta2 =
step =

(1 - )

In [1]:
@tf.keras.utils.register_keras_serializable(package="Addons")
class RectifiedAdam(tf.keras.optimizers.Optimizer):
    def __init__(self):
        super(RectifiedAdam, self).__init__()


        self._set_hyper()
        self._decayed_lr()


    # Optimizer 상속 받은 메서드들 활용 add_slot, set_hyper
    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, "m")
        for var in var_list:
            self.add_slot(var, "v")
        if self.amsgrad:
            for var in var_list:
                self.add_slot(var, "vhat")


SyntaxError: invalid syntax (4275515357.py, line 1)

In [ ]:
class RectifiedAdam(tf.keras.optimizers.Optimizer):

    def __init__(self):
        super().__init__(name, **kwargs)

    # m_zero, v_zero moving 1st, 2nd moment
    # p_inf <- 2/(1-beta2)-1  Simple Moving-Average 의 최대 길이를 계산
    from typing import List

    def _single_tensor_radam(self , params:List[tf.Tensor]
                                   , grads:List[tf.Tensor]
                                   , exp_avgs:List[tf.Tensor]
                                    , exp_avg_sqs:List[tf.Tensor]
                                    ,state_steps:List[tf.Tensor]
                                    ,*
                             , beta1 :tf.float16
                             , beta2  :tf.float16
                             ,lr : tf.float16
                             ,weight_decay: tf.float
                             ,eps:float ):

        for i, param in enumerate(params):
            grads = grads[i]
            exp_avg = exp_avgs[i]
            exp_avg_sq = exp_avg_sqs[i]
            step_t = state_steps[i]

            # update step
            step_t +=1
            # tensor item
            step = step_t.items()

            # beta2  (1-beta2)*(grads**2)
            # gradient = func*(param*step)
            # v_zero  = beta2*(v_zero*step) + (1-beta2)*(grad*step **2) 이전 시점 v_zero

            # 최초  exp_avg (지수이동평균값을 parameter로 받으므로 받은 후 exp_avg t시점은 t+1 시점을 만든다)

            bias_correction1 = 1-beta1**step
            bias_correction2=  1- beta2 **step

            if weight_decay != 0:
                # torch : grad = grad.add(param, alpha=weight_decay)
                grad =  grad+ (weight_decay*param)


            exp_avg = tf.math.multiply(exp_avg,beta1) + ( 1- beta1)*grads
            """
             m = self.get_slot(var, "m")
            m_scaled_g_values = grad * coef["one_minus_beta_1_t"]
            m_t = m.assign(m * coef["beta_1_t"], use_locking=self._use_locking)
            with tf.control_dependencies([m_t]):
                m_t = self._resource_scatter_add(m, indices, m_scaled_g_values)
            m_corr_t = m_t * coef["recip_one_minus_beta_1_power"]

            """

            exp_avg_sq = tf.math.multiply( exp_avg_sq*beta2) + (1-beta2) * tf.math.multiply(gradsgrads)
            """
            이 한줄과 같은 코드인데 slot 을 가져오고 앞의 항과 뒤의 항을 따로 만든 뒤
            control_depenencies context 내에서 _resource_scatter_add 라는 메서드로 + 를 해야하는 듯 하다
            v = self.get_slot(var, "v")
            v_scaled_g_values = (grad * grad) * coef["one_minus_beta_2_t"]
            v_t = v.assign(v * coef["beta_2_t"], use_locking=self._use_locking)
            with tf.control_dependencies([v_t]):
                v_t = self._resource_scatter_add(v, indices, v_scaled_g_values)
            """

            # Decay the first and second moment running average coefficient
            #bias_correction = bias_correction / (1-beta1**eps)

            # decay rate에 step을 제곱하여 1에서 뺀값으로 나누어
            # 지수이동평균을 나누어 편향을 수정한다
            bias_corrected_exp_avgs = exp_avgs/bias_correction1
            # 지수이동평균 추정치 길이의 최대값
            p_inf = 2/(1-beta2)-1

            # compute the length of the approximated SMA 지수이동평균 길이의 추정치를 계산
            p_t =  p_inf-2*step*(beta2**step) / bias_correction2

            #
#            bias_correction =

            # 음수 값이 되지 않는선
            if p_t > 5:
                # Compute adaptive learning rate
                lr = tf.math.sqrt(bias_correction2)/tf.math.sqrt(exp_avg_sqs+eps )
                # Comput the variance reccitfication term
                rectification_term = tf.math.sqrt((p_t -4)(p_t-2) *p_inf / ((p_inf-4)(p_inf-2)*p_t))

                # add_(input, other, alpha) -> input+ (other*alpha)
                # param.add_(bias_corrected_exp_avg * lr , alpha=-1.0)
                # === param - bias_corrected_exp_avg *lr
                alpha=-1
                param=  param -( rectification_term * bias_corrected_exp_avgs*lr*alpha)
                # 이것 자체로 param 은 t시점이된다
            else:
                param=  param -( bias_corrected_exp_avgs*lr*alpha)

"""
tensorflow 구현
 def _resource_apply_dense(self, grad, var, apply_state=None):
        var_device, var_dtype = var.device, var.dtype.base_dtype
        coef = (apply_state or {}).get(
            (var_device, var_dtype)
        ) or self._fallback_apply_state(var_device, var_dtype)
        m = self.get_slot(var, "m")
        v = self.get_slot(var, "v")

        m_t = m.assign(
            coef["beta_1_t"] * m + coef["one_minus_beta_1_t"] * grad,
            use_locking=self._use_locking,
        )
        m_corr_t = m_t * coef["recip_one_minus_beta_1_power"]

        v_t = v.assign(
            coef["beta_2_t"] * v + coef["one_minus_beta_2_t"] * tf.square(grad),
            use_locking=self._use_locking,
        )
        if self.amsgrad:
            vhat = self.get_slot(var, "vhat")
            vhat_t = vhat.assign(tf.maximum(vhat, v_t), use_locking=self._use_locking)
            v_corr_t = tf.sqrt(vhat_t * coef["recip_one_minus_beta_2_power"])
        else:
            vhat_t = None
            v_corr_t = tf.sqrt(v_t * coef["recip_one_minus_beta_2_power"])

        var_t = tf.where(
            coef["sma_t_ge_sma_threshold"],
            coef["r_t"] * m_corr_t / (v_corr_t + coef["epsilon_t"]),
            m_corr_t,
        )

        if self._has_weight_decay:
            var_t += coef["wd_t"] * var

        var_update = var.assign_sub(coef["lr_t"] * var_t, use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]
        if self.amsgrad:
            updates.append(vhat_t)
        return tf.group(*updates)



"""
"""
pytorch 구현
def _single_tensor_radam(params: List[Tensor],
                         grads: List[Tensor],
                         exp_avgs: List[Tensor],
                         exp_avg_sqs: List[Tensor],
                         state_steps: List[Tensor],
                         *,
                         beta1: float,
                         beta2: float,
                         lr: float,
                         weight_decay: float,
                         eps: float):

    for i, param in enumerate(params):
        grad = grads[i]
        exp_avg = exp_avgs[i]
        exp_avg_sq = exp_avg_sqs[i]
        step_t = state_steps[i]
        # update step
        step_t += 1
        step = step_t.item()

        bias_correction1 = 1 - beta1 ** step
        bias_correction2 = 1 - beta2 ** step

        if weight_decay != 0:
            grad = grad.add(param, alpha=weight_decay)

        # Decay the first and second moment running average coefficient
        exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
        exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

        # correcting bias for the first moving moment
        bias_corrected_exp_avg = exp_avg / bias_correction1

        # maximum length of the approximated SMA
        rho_inf = 2 / (1 - beta2) - 1
        # compute the length of the approximated SMA
        rho_t = rho_inf - 2 * step * (beta2 ** step) / bias_correction2

        if rho_t > 5.:
            # Compute the variance rectification term and update parameters accordingly
            rect = math.sqrt((rho_t - 4) * (rho_t - 2) * rho_inf / ((rho_inf - 4) * (rho_inf - 2) * rho_t))
            adaptive_lr = math.sqrt(bias_correction2) / exp_avg_sq.sqrt().add_(eps)

            param.add_(bias_corrected_exp_avg * lr * adaptive_lr * rect, alpha=-1.0)
        else:
            param.add_(bias_corrected_exp_avg * lr, alpha=-1.0)


"""